# Mezei flipper I

In [ ]:
import numpy as np
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from qutip import *
from scipy.constants import physical_constants

# Modules for visualization 
import os
import subprocess
from IPython import display

In [ ]:
# Define and import some constants
h_bar = physical_constants["Planck constant over 2 pi in eV s"][0] #h_bar = ℏ = 6.582119514e-16 (Ev*S) 
gyr_n = physical_constants["neutron gyromag. ratio"][0] #γ= 1.825*10^8 s⁻¹T⁻¹ 
l = 0.01 #flight path length [m]
v = 1000 #neutron velocity [m/s]

In [ ]:
def folder(name="mezei_tmp"):
    """
    Checks if a folder with the given name exists. If it does not, the folder is created.
    Parameters:
    name (str): The name of the folder to check and potentially create.
    """
    if not os.path.exists(name):
        os.makedirs(name)

def create_video(path, name, fps=30):
    command = f'ffmpeg -framerate {fps} -i {path}/frame_%d.png -c:v libx264 -pix_fmt yuv420p {name}.mp4'
    subprocess.run(command, shell=True, check=True)

def create_gif(path, name):
    command = f'ffmpeg -framerate 30 -i {path}/frame_%d.png {name}.gif'
    subprocess.run(command, shell=True, check=True)

def display_gif(fn):
    return display.HTML('<img src="{}">'.format(fn))

def display_video(video_path):
    return display.Video(video_path)

In [ ]:
#H = ((np.pi*v)/l)*(sigmay()) + ((np.pi*v)/l)*(sigmaz())

coef_h = (np.pi*v)/l
H = coef_h*(sigmay() + sigmaz())
psi0 = (basis(2, 0) + 1j * basis(2, 1)).unit() # Initial state |+i> or just y-axis
times = np.linspace(0.0, (np.pi*l)/(v*9), 105)
result = sesolve(H, psi0, times, [])

In [ ]:
folder()

In [ ]:
cmap = plt.get_cmap('seismic')
num_of_points = len(times)
colors = [cmap(i/num_of_points) for i in range(num_of_points)]

b = Bloch()

b.zlabel = ['|↑⟩', '|↓⟩']
b.point_marker = ['o']  # Set all point markers to circles
b.point_size = [20]
b.vector_color = ['green']
b.point_color = colors

n = 0
for i in range(len(result.states)):
    b.add_states(result.states[i], kind='vector')
    b.save(f'mezei_tmp/frame_{n}.png', format='png')
    b.add_states(result.states[i], kind='point')
    b.save(f'mezei_tmp/frame_{n+1}.png', format='png')
    b.clear()
    for j in range(i+1):
        b.add_states(result.states[j], kind='point')
    n+=2

In [ ]:
create_gif('mezei_tmp', 'mezei')